In [66]:
import plotly as py
import cufflinks as cf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import datetime as dt
import math
from random import *
from dateutil.parser import parse
warnings.filterwarnings("ignore")

plt.style.use('ggplot')
print(cf.__version__)

# %config InlineBackend.figure_format = 'retina'
cf.go_offline()

0.17.0


In [82]:
dt_index = pd.DataFrame(pd.date_range(start='20100101', end='20191231')).rename(columns = {0 : 'date'})

def processing_Exchange_Rate(data):
    print('start')
    # 20100101부터 정렬
    data = data[::-1]
    # format datetime으로 변경
    data['날짜'] = data['날짜'].apply(lambda x : parse(x[:4] + x[6:8] + x[10:12]))
    # 20100101-20191231 인덱스 설정
    data = pd.merge(dt_index, data, how = 'left', left_on='date', right_on='날짜').set_index('date')
    print('전처리 전 null 갯수 : {}'.format(data['종가'].isnull().sum()))
    # type 변경 str -> float
    data['종가'] = data['종가'].apply(lambda x : x if x is np.nan else float(x.replace(',','')))
    data['오픈'] = data['오픈'].apply(lambda x : x if x is np.nan else float(x.replace(',','')))
    
    # 주말 Null값 채우기
    # 일요일 : 월요일 오픈 가격으로 대체
    data['Fill_value'] = data.shift(-1)['오픈']
    data['종가'] = data[['종가', 'Fill_value']].apply(lambda x : x['Fill_value'] if math.isnan(x['종가']) else x, axis = 1)
    print('전처리 후 null 갯수 : {}'.format(data['종가'].isnull().sum()))
    # 토요일 : 금요일 종가 + random value
    random_value = 3
    data['Fill_value'] = data.shift(1)['종가']
    data['종가'] = data[['종가', 'Fill_value']].apply(lambda x : x['Fill_value'] + uniform(-random_value, random_value) if math.isnan(x['종가']) else x, axis = 1)

    data.drop(['날짜', '오픈', '고가', '저가', '변동 %', 'Fill_value'], axis = 1, inplace = True)    
                      
    print('전처리 후 null 갯수 : {}'.format(data['종가'].isnull().sum()))
    
    print('end')
    return data

In [83]:
Exchange_Rate = processing_Exchange_Rate(pd.read_csv('USD_KRW.csv'))
Exchange_Rate.head(5)

start
전처리 전 null 갯수 : 1044
전처리 후 null 갯수 : 522
전처리 후 null 갯수 : 0
end


,종가
date,
2010-01-01,1166.080000
2010-01-02,1163.278731
2010-01-03,1164.070000
2010-01-04,1150.000000
2010-01-05,1140.200000
